In [1]:
import pandas as pd
import numpy as np
import base64
import pyodbc
import os

In [2]:
# Create Connection
password = config.password
# Connecting to SQL Server
client_code = '544'
database = 'SS'
base_dir = r"C:\Users\jesse.henson\StaffScapes\StaffScapes - Documents\IT\Isolved Import\EE_upload"
conn = pyodbc.connect('Driver={SQL Server Native Client 10.0};'
                      'Server=staffc-darwin\darwin12;'
                      f'Database={database};'
                      'UID=username;'
                      'PWD=' + password + ';')
if not os.path.isdir(f'{base_dir}/{client_code}'):
    os.mkdir(f'{base_dir}/{client_code}')
# paycode_data = pd.read_sql_query('Select * from upr30300',conn,parse_dates=['PeriodBeginDate','PeriodEndDate','CheckDate'])
# paycode_data

In [3]:

sql_str = '''
select format(upr00100.STRTDATE,'MM/dd/yyyy') as AdjustedServiceDate, 
upr00100.EMPLOYID as EmployeeNumber, 
upr00100.FRSTNAME as FirstName, 
upr00100.LASTNAME as LastName, 
TW00824.DefaultEmailAddress AS PersonalEmail,
TW00824.DefaultEmailAddress AS SelfServiceEmail,
upr00102.ADDRESS1 as Address1, 
upr00102.ADDRESS2 as Address2,
upr00102.ZIPCODE as ZipCode, 
upr00102.CITY as City, 
upr00102.state as State, 
'26' as Frequency, 
EMPLOYMENTTYPE as EmploymentCategory, 
upr00100.DEMPINAC as StatusDate,
'Corp' as WorkLocation,
ZIPCODE as ResidentLocation,
Deprtmnt as #department,
UPR00100.SOCSCNUM AS SSN,
'SSN' AS IDType,
format(tw00820.ThinkwareOriginHireDate, 'MM/dd/yyy') as HireDate,
upr00100.BRTHDATE as BirthDate,
upr00102.PHONE1 as HomePhone,
upr00100.ETHNORGN as EthnicOrigin,
upr00100.GENDER as Gender,
upr00100.WRKRCOMP as WorkersComp,
upr00100.INACTIVE as Status
from upr00100
join tw00819 on upr00100.EMPLOYID = tw00819.EMPLOYID
left outer join tw00820 on upr00100.employid = tw00820.employid
left outer join upr00102 on upr00100.EMPLOYID = upr00102.EMPLOYID and upr00100.ADRSCODE = upr00102.ADRSCODE
left outer JOIN TW00824 ON UPR00100.EMPLOYID = TW00824.EMPLOYID
'''

where_claus = f"where CUSTNMBR = '{client_code}'"

employee_main = pd.read_sql_query(sql_str + where_claus,conn, parse_dates=['BirthDate'])
employee_main['BirthDate'] = employee_main['BirthDate'].dt.strftime('%m/%d/%Y')
employee_main['Gender'] = employee_main['Gender'].map({1:'M',2:'F'} )
employee_main['EthnicOrigin'] = employee_main['EthnicOrigin'].map({1:2, 7:8, 5:1, 6:7, 3:3, 2:6, 8:4, 4:5})
employee_main['EthnicOrigin'] = employee_main['EthnicOrigin'].fillna(7)
employee_main['EthnicOrigin'] = employee_main['EthnicOrigin'].apply(lambda x: int(x))
employee_main['EmployeeNumber'] = employee_main['EmployeeNumber'].apply(lambda x:x[2:10])
employee_main['EmploymentCategory'] = employee_main['EmploymentCategory'].map({0: 'FT', 1: 'FT', 2: 'PT', 3:'FT-T', 4:'PT-T'})
employee_main['Status'] = employee_main['Status'].map({0: 'A', 1: 'T'})
employee_main['HomePhone'] = employee_main['HomePhone'].apply(lambda x:x[0:10])
# employee_main['SelfServiceEnabled'] = employee_main['Status'].apply(lambda x: "Y" if x== 'A' else '')
employee_main['WorkersComp'] = employee_main['WorkersComp'].apply(lambda x: x[2:])
# employee_main['EmploymentCategory'] = employee_main['EmploymentCategory'].apply(lambda x:'FT' if x == 'FT-T' else 'PT')
employee_main['EmploymentCategory']

0     FT
1     FT
2     FT
3     FT
4     FT
      ..
76    FT
77    FT
78    FT
79    FT
80    FT
Name: EmploymentCategory, Length: 81, dtype: object

In [4]:
sql_str = '''
SELECT EMPLOYID as EmployeeNumber, PAYRCORD as PayType, PAYRTAMT as HourlyRate FROM upr00400
'''


where_claus = f"where PAYRCORD in ('Salary','hourly') and EMPLOYID BETWEEN '01{client_code}0001' and '01{client_code}9999'"

rate_main = pd.read_sql_query(sql_str + where_claus,conn)
rate_main['EmployeeNumber'] = rate_main['EmployeeNumber'].apply(lambda x:x[2:10])
rate_main['PerPaySalary'] = rate_main['HourlyRate'].apply(lambda x: x if x > 1000 else 0 )
rate_main['HourlyRate'] = rate_main['HourlyRate'].apply(lambda x: 0 if x > 1000 else x)
rate_main

,EmployeeNumber,PayType,HourlyRate,PerPaySalary
0,5440091,HOURLY,15.0,0.0
1,5440093,HOURLY,15.0,0.0
2,5440094,HOURLY,15.0,0.0
3,5440092,HOURLY,15.0,0.0
4,5440085,HOURLY,14.0,0.0
...,...,...,...,...
81,5440099,HOURLY,21.5,0.0
82,5440098,HOURLY,16.0,0.0
83,5440100,HOURLY,18.5,0.0
84,5440095,HOURLY,25.0,0.0


In [5]:
sql_str = '''
SELECT
upr00300.EMPLOYID as EmployeeNumber, 
UPR00300.FDFLGSTS AS FedFilingStatus,
upr00300.FEDEXMPT as FedExemptions,
upr00300.ADFDWHDG as FedAddlAmount,
TWUPR003.DependentClaimAmt as FedDependentsAmt,
upr00700.TXFLGSTS as StateFilingStatus,
upr00700.EXMTAMNT as StateExemptionAmount,
upr00700.ADSTWHDG as StateAddlAmount,
upr00700.ADNLALOW as StateAddlExemptions FROM UPR00300
JOIN TWUPR003 ON UPR00300.EMPLOYID = TWUPR003.EMPLOYID
join upr00700 on upr00300.EMPLOYID = upr00700.EMPLOYID
'''

where_claus = f"where upr00300.EMPLOYID BETWEEN '01{client_code}0001' and '01{client_code}9999'"


tax_main = pd.read_sql_query(sql_str + where_claus, conn)
tax_main['EmployeeNumber'] = tax_main['EmployeeNumber'].apply(lambda x:x[2:10])
tax_main['FedFilingStatus'] = tax_main['FedFilingStatus'].apply(lambda x: 'FDM' if x.strip() == 'MAR' else 'FDS')
tax_main['StateFilingStatus'] = tax_main['StateFilingStatus'].apply(lambda x: 'COM' if x.strip() == 'MAR' else 'COS')
tax_main['StateAddlAmount'] = tax_main['StateAddlAmount'].apply(lambda x: 0 if x < 0 else x)
tax_main

,EmployeeNumber,FedFilingStatus,FedExemptions,FedAddlAmount,FedDependentsAmt,StateFilingStatus,StateExemptionAmount,StateAddlAmount,StateAddlExemptions
0,5440005,FDS,0,0.0,0.0,COS,0.0,0.0,0
1,5440047,FDS,1,0.0,0.0,COS,0.0,0.0,0
2,5440013,FDM,0,0.0,0.0,COM,0.0,0.0,0
3,5440036,FDS,0,0.0,0.0,COS,0.0,0.0,0
4,5440019,FDS,0,0.0,0.0,COS,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...
76,5440095,FDS,2,0.0,0.0,COS,0.0,0.0,0
77,5440020,FDM,3,0.0,0.0,COM,0.0,0.0,0
78,5440033,FDM,0,0.0,0.0,COM,0.0,0.0,0
79,5440032,FDS,0,0.0,0.0,COS,0.0,0.0,0


In [6]:
temp_df = employee_main.merge(rate_main, 'left', on='EmployeeNumber')
ee_full = temp_df.merge(tax_main, 'left', on='EmployeeNumber')
ee_full['PayType'] = ee_full['PayType'].fillna('HOURLY')
ee_full['PayType'] = ee_full['PayType'].replace('','HOURLY')
ee_full['PayType'] = ee_full['PayType'].apply(lambda x: 'HOURLY' if str(x).strip() == '' else x)
ee_full.to_csv(f'{base_dir}/{client_code}/ee_full.csv')
ee_full['PayType']

0     HOURLY 
1     HOURLY 
2     HOURLY 
3     HOURLY 
4     HOURLY 
       ...   
81    HOURLY 
82    HOURLY 
83    HOURLY 
84    HOURLY 
85    HOURLY 
Name: PayType, Length: 86, dtype: object

In [7]:

sql_str = '''
select tw00107.EMPLOYID AS [Key], ThinkInactive as ActiveStatus, ThinkPrecheck as PrecheckStatus, ThkAcctDesc as AccountType, ThinkDirDepOpt as Sequence, 'Y' as Frequency, 
ThinkwarePerc as [Percent], ThkDirDepAmt as Amount, ThkAcctNo as AccountNumber, ThinkBanks as RoutingNumber
from TW00107
JOIN UPR00100 on upr00100.employid = tw00107.employid
'''

where_claus = f"where tw00107.EMPLOYID BETWEEN '01{client_code}0001' and '01{client_code}9999' and upr00100.inactive = 0"

dd_main = pd.read_sql_query(sql_str + where_claus, conn)
dd_main['Key'] = dd_main['Key'].apply(lambda x:x[2:10])
dd_main['AccountType'] = dd_main['AccountType'].apply(lambda x: 'C' if x.strip() == 'CHECKING' else 'S')
dd_main['Status'] = dd_main['ActiveStatus'] * 10 + dd_main['PrecheckStatus'] * 5
dd_main['Status'] = dd_main['Status'].map({0:'A', 5:'P', 10:'I', 15:'I'})
dd_main['Sequence'] = dd_main['Sequence'].apply(lambda x: 0 if x==4 else 1)
dd_main['Amount'] = dd_main['Amount'].replace(0,'')
dd_main['Percent'] = dd_main['Percent'].replace(0,'')
dd_main.to_csv(f'{base_dir}/{client_code}/dd_main.csv')
dd_main

,Key,ActiveStatus,PrecheckStatus,AccountType,Sequence,Frequency,Percent,Amount,AccountNumber,RoutingNumber,Status
0,5440008,0,0,C,0,Y,,,154512488045,071904779,A
1,5440013,0,0,C,0,Y,,,9915937289,102000076,A
2,5440013,1,0,C,0,Y,,,760642066095,307070267,I
3,5440013,1,0,C,0,Y,,,766921480683,307070267,I
4,5440015,0,0,C,0,Y,,,4501303567,107005047,A
5,5440020,0,0,C,0,Y,,,415142053,102001017,A
6,5440023,0,0,C,0,Y,,,3622508145,102000076,A
7,5440024,0,0,C,0,Y,,,139076510,102001017,A
8,5440026,0,0,C,0,Y,,,1080002447910,302075128,A
9,5440027,0,0,C,0,Y,,,156155386980,031101279,A


In [8]:
sql_str = '''
select EMPLOYID as [Key], BENEFIT as PayItem, BNFPRCNT_1 as [Percent], BNFITAMT_1 as Amount from upr00600
'''

where_claus = f"where EMPLOYID BETWEEN '01{client_code}0001' and '01{client_code}9999' and Inactive = 0"

ben_main = pd.read_sql_query(sql_str + where_claus, conn)
ben_main['Key'] = ben_main['Key'].apply(lambda x:x[2:10])
ben_main = ben_main[ben_main['PayItem'].str.contains('TAX') == False]
ben_main = ben_main[ben_main['PayItem'].str.contains('401') == False]
ben_main['ParamType'] = ben_main['Percent'].apply(lambda x: "Percent" if x > 0 else "Amount")
ben_main['ParamValue'] = ben_main['Percent'] + ben_main['Amount']
ben_main.to_csv(f'{base_dir}/{client_code}/ben_main.csv')
ben_main

,Key,PayItem,Percent,Amount,ParamType,ParamValue
23,5440023,MED,0.0,0.01,Amount,0.01


In [9]:
sql_str = '''
select EMPLOYID as [Key], DEDUCTON as PayItem, DEDNPRCT_1 as [Percent], DEDCAMNT_1 as Amount from upr00500
'''

where_claus = f"where EMPLOYID BETWEEN '01{client_code}0001' and '01{client_code}9999' and Inactive = 0"

ded_main = pd.read_sql_query(sql_str + where_claus, conn)
ded_main['Key'] = ded_main['Key'].apply(lambda x:x[2:10])
ded_main = ded_main[ded_main['PayItem'].str.contains('TAX') == False]
ded_main = ded_main[ded_main['PayItem'].str.contains('401') == False]
ded_main = ded_main[ded_main['PayItem'].str.contains('SIM') == False]
ded_main = ded_main[ded_main['PayItem'].str.contains('CHILD') == False]
ded_main = ded_main[ded_main['PayItem'].str.contains('GRN') == False]
ded_main['ParamType'] = ded_main['Percent'].apply(lambda x: "Percent" if x > 0 else "Amount")
ded_main['ParamValue'] = ded_main['Percent'] + ded_main['Amount']
ded_main.to_csv(f'{base_dir}/{client_code}/ded_main.csv')
ded_main

,Key,PayItem,Percent,Amount,ParamType,ParamValue
1,5440089,DEDUCT,0.0,25.00,Amount,25.00
2,5440089,VIS-1,0.0,1.50,Amount,1.50
3,5440089,DNT-1,0.0,4.51,Amount,4.51
4,5440086,DNT-1,0.0,4.51,Amount,4.51
5,5440086,VIS-1,0.0,1.50,Amount,1.50
6,5440086,AC-CI,0.0,13.47,Amount,13.47
8,5440038,DEDUCT,0.0,25.00,Amount,25.00
11,5440073,DEDUCT,0.0,25.00,Amount,25.00
19,5440053,AC-CI,0.0,20.96,Amount,20.96
20,5440024,CL MD1,0.0,169.25,Amount,169.25


In [10]:
sql_str = '''
select right(rtrim(EMPLOYID), 7) as [Key], DSCRIPTN as [Plan], TW_AvailableHours as Balance from tw80514
'''

where_claus = f"where EMPLOYID BETWEEN '01{client_code}0001' and '01{client_code}9999' and Inactive = 0"
acc_main = pd.read_sql_query(sql_str + where_claus, conn)
acc_main = pd.read_sql_query(sql_str + where_claus, conn)
acc_main['Balance'] = acc_main['Balance'] * .01
acc_main.to_csv(f'{base_dir}/{client_code}/accrual_main.csv')
acc_main

,Key,Plan,Balance
